In [1]:
import gzip
import json
import nibabel as nib
import numpy as np
import os
import pickle
import sys
import time
import torch

sys.path.append('./')
sys.path.append('./sw/')

from IVIMNET.hyperparams import hyperparams as hp_example_1
import IVIMNET.simulations as sim
import IVIMNET.deep as deep
import IVIMNET.deep_bayes as deep_bayes
import IVIMNET.fitting_algorithms as fit

arg = hp_example_1()
arg = deep.checkarg(arg)
arg.fit.do_fit = False

testdata = False

EPOCHS = 1
BAYES_SAMPLES = 32

### folder patient data
folder = './sw/data/example_data'
model_path = './sw/models/bnn_e09_ivim_patient'

./sw/IVIMNET/deep.py:882: UserWarning: arg no norm_data_full. Using default of False
  warnings.warn('arg no norm_data_full. Using default of False')
./sw/IVIMNET/deep.py:789: UserWarning: arg.train.plateau_size not defined. Using default of 20
  warnings.warn('arg.train.plateau_size not defined. Using default of 20')


In [2]:
### load patient data
print('Load patient data \n')
# load and init b-values
text_file = np.genfromtxt('{folder}/bvalues.bval'.format(folder=folder))
bvalues = np.array(text_file)
selsb = np.array(bvalues) == 0
# load nifti
data = nib.load('{folder}/data.nii.gz'.format(folder=folder))
datas = data.get_fdata() 
# reshape image for fitting
sx, sy, sz, n_b_values = datas.shape 
X_dw = np.reshape(datas, (sx * sy * sz, n_b_values))

### select only relevant values, delete background and noise, and normalise data
S0 = np.nanmean(X_dw[:, selsb], axis=1)
S0[S0 != S0] = 0
S0 = np.squeeze(S0)
valid_id = (S0 > (0.5 * np.median(S0[S0 > 0]))) 
datatot = X_dw[valid_id, :]
# normalise data
S0 = np.nanmean(datatot[:, selsb], axis=1).astype('<f')
datatot = datatot / S0[:, None]
print('Patient data loaded\n')

print('NN fitting\n')
res = [i for i, val in enumerate(datatot != datatot) if not val.any()] # Remove NaN data

Load patient data 

Patient data loaded

NN fitting



In [3]:
arg = hp_example_1()
arg = deep.checkarg(arg)
arg.fit.do_fit = False
arg.train_pars.batch_size = 32
net_params = deep_bayes.net_params()

./sw/IVIMNET/deep.py:882: UserWarning: arg no norm_data_full. Using default of False
  warnings.warn('arg no norm_data_full. Using default of False')
./sw/IVIMNET/deep.py:789: UserWarning: arg.train.plateau_size not defined. Using default of 20
  warnings.warn('arg.train.plateau_size not defined. Using default of 20')


In [4]:
time_start = time.perf_counter()
net, epochs, best_val_loss = deep_bayes.learn_IVIM(datatot[res], bvalues, arg, net_params=net_params, epochs=EPOCHS, stats_out=True, bayes_samples=BAYES_SAMPLES)
time_end = time.perf_counter() - time_start
print('\ntime elapsed for Net: {}\n'.format(time_end))

-----------------------------------------------------------------
Epoch: 0; Bad epochs: 0


501it [00:14, 35.33it/s]                           


Validation


100%|██████████| 10/10 [00:00<00:00, 47.08it/s]

Validation loss: 0.050373968482017514
############### Saving good model ###############################

Done

time elapsed for Net: 18.313867077231407



In [5]:
net.eval()

Net(
  (fc_layers): ModuleList(
    (0): ModuleList(
      (0): Linear(in_features=104, out_features=104, bias=True)
      (1): BernoulliDropout(
        p=0.10000000149011612, quant=False
        (mul_mask): FloatFunctional(
          (activation_post_process): Identity()
        )
        (mul_scalar): FloatFunctional(
          (activation_post_process): Identity()
        )
      )
      (2): BatchNorm1d(104, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU()
      (4): Linear(in_features=104, out_features=104, bias=True)
      (5): BernoulliDropout(
        p=0.10000000149011612, quant=False
        (mul_mask): FloatFunctional(
          (activation_post_process): Identity()
        )
        (mul_scalar): FloatFunctional(
          (activation_post_process): Identity()
        )
      )
      (6): BatchNorm1d(104, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (7): ReLU()
    )
    (1): ModuleList(
      (0): Linear(in_feature

In [6]:
datatot.shape

(110589, 104)

In [18]:
torch.cuda.empty_cache()

In [19]:
infer_data = torch.from_numpy(datatot.astype(np.float32)).to("cuda")
%timeit cpu_net(infer_data)

RuntimeError: CUDA out of memory. Tried to allocate 1.37 GiB (GPU 0; 10.92 GiB total capacity; 132.00 MiB already allocated; 1.24 GiB free; 134.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
net